In [1]:
import pandas as pd
import pickle

In [3]:
dataset = pd.read_csv("../50_Startups.csv")
dataset = pd.get_dummies(dataset, drop_first=True)
dataset = dataset.astype(int)
dataset

,R&D Spend,Administration,Marketing Spend,Profit,State_Florida,State_New York
0,165349,136897,471784,192261,0,1
1,162597,151377,443898,191792,0,0
2,153441,101145,407934,191050,1,0
3,144372,118671,383199,182901,0,1
4,142107,91391,366168,166187,1,0
5,131876,99814,362861,156991,0,1
6,134615,147198,127716,156122,0,0
7,130298,145530,323876,155752,1,0
8,120542,148718,311613,152211,0,1
9,123334,108679,304981,149759,0,0


In [5]:
dataset.columns

Index(['R&D Spend', 'Administration', 'Marketing Spend', 'Profit',
       'State_Florida', 'State_New York'],
      dtype='object')

In [7]:
independent = dataset[['R&D Spend', 'Administration', 'Marketing Spend',
       'State_Florida', 'State_New York']]
dependent = dataset['Profit']

In [9]:
#split Training and Test datasets
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(independent, dependent, test_size=0.30, random_state = 0)

In [11]:
#Standardisation (Defined range of datasets) the Train and Test data
from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
xTrain = scX.fit_transform(xTrain)
xTest = scX.transform(xTest)

# Save the fitted scaler
with open('scalerX.pkl', 'wb') as f:
    pickle.dump(scX, f)

In [13]:
import numpy as np
scY = StandardScaler()
# Convert Series to NumPy array, then reshape
yTrain_np = yTrain.values.reshape(-1, 1)
yTest_np = yTest.values.reshape(-1, 1)

# Fit and transform
yTrain_scaled = scY.fit_transform(yTrain_np)
yTest_scaled = scY.transform(yTest_np)
# Save the fitted scaler
with open('scalerY.pkl', 'wb') as f:
    pickle.dump(scY, f)

In [15]:
#model creation
from sklearn.svm import SVR
regressor = SVR(kernel="linear")

regressor.fit(xTrain, yTrain)

SVR(kernel='linear')

In [17]:
regressor.n_support_

array([34], dtype=int32)

In [19]:
regressor.support_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34],
      dtype=int32)

In [21]:
yPred = regressor.predict(xTest)

In [23]:
#Metrics validation using r2 score function
from sklearn.metrics import r2_score
rScore = r2_score(yTest, yPred)
rScore

-0.055698007950230055

In [25]:
fileName = "Final_SVMR_PreBoth_Model.sav"
pickle.dump(regressor, open(fileName, 'wb'))

In [27]:
preInput = scX.transform([[1300,12000,4000,0,1]])
preInput

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-1.46754484, -4.33834816, -1.50743981, -0.5       ,  1.30088727]])

In [29]:
loaded_model=pickle.load(open("Final_SVMR_PreBoth_Model.sav", 'rb'))
result=loaded_model.predict(preInput)
result

array([108603.27619641])

In [ ]:
preOutput = scY.inverse_transform(result.reshape(-1, 1))
preOutput